<a href="https://colab.research.google.com/github/GiX007/agent-labs/blob/main/03_langChain/00_template_prompts_and_output_parsers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain: Models, Prompts and Output Parsers

## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [ ]:
#!pip install python-dotenv
#!pip install openai

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
dotenv_path = find_dotenv() or '/content/OPENAI_API_KEY.env' # read local .env file
load_dotenv(dotenv_path)

openai_api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI(api_key=openai_api_key)

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers.

In [ ]:
# Set the model variable based on the best and cheapest available choice at the current date
llm_model = "gpt-4o-mini"

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [ ]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

In [ ]:
get_completion("What is 1+1?")

'1 + 1 equals 2.'

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
"""

In [ ]:
style = """American English in a calm and respectful tone"""

In [ ]:
prompt = f"""Translate the text that is delimited by triple backticks into a style that is {style}.
text: ```{customer_email}```"""

print(prompt)

Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone. 
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```


In [ ]:
response = get_completion(prompt)

In [ ]:
response

'I’m quite frustrated that the lid of my blender came off and splattered my kitchen walls with smoothie. To make matters worse, the warranty doesn’t cover the cost of cleaning up my kitchen. I would really appreciate your assistance with this issue. Thank you!'

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [ ]:
!pip install langchain langchain-openai

In [ ]:
import langchain
print("LangChain version:", langchain.__version__)

LangChain version: 0.3.27


### Model

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
# To control the randomness and creativity of the generated text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7e813f89e930>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7e813f4e3bc0>, root_client=<openai.OpenAI object at 0x7e813fba04a0>, root_async_client=<openai.AsyncOpenAI object at 0x7e813f89e9f0>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

### Prompt template

In [ ]:
template_string = """Translate the text that is delimited by triple backticks into a style that is {style}.
text: ```{text}```
"""

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# create the first template
prompt_template = ChatPromptTemplate.from_template(template_string)

In [ ]:
prompt_template

ChatPromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}.\ntext: ```{text}```\n'), additional_kwargs={})])

In [ ]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}.\ntext: ```{text}```\n')

In [ ]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

Having created a prompt template  with 2 input variables, let's try it for a simple example.

Example 1.

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
"""

In [ ]:
customer_style = """American English in a calm and respectful tone"""

In [ ]:
# notice how we define the new message by using 'format_messages' method of the template
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [ ]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [ ]:
print(customer_messages)

[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone.\ntext: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n", additional_kwargs={}, response_metadata={})]


In [ ]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone.\ntext: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


In [ ]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

In [ ]:
print(customer_response.content)

I am quite frustrated that the lid of my blender came off and splattered smoothie all over my kitchen walls. To make matters worse, the warranty does not cover the cost of cleaning up my kitchen. I would appreciate your assistance with this issue. Thank you.


Translate Text into a Friendly Customer Tone (LangChain + GPT-4o-mini) - The simplest workflow

```python
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 1. Initialize model
chat = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# 2. Define prompt
prompt = ChatPromptTemplate.from_template(
    "Translate this message into a friendly customer tone: {message}"
)

# 3. Format input
customer_messages = [prompt.format_messages(message="I need a refund right now!")]

# 4. Invoke model
response = chat.invoke(customer_messages[0])

# 5. Print output
print(response.content)

Example 2.

In [ ]:
service_reply = """Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
"""

In [ ]:
service_style_pirate = """a polite tone that speaks in English Pirate"""

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate.
text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [ ]:
service_response = chat.invoke(service_messages)
print(service_response.content)

Ahoy, me hearty! I be sorry to inform ye that the warranty be not coverin' the costs of cleanin' yer galley, as it seems ye may have misused yer blender by forgettin' to secure the lid afore settin' it to whirl. Aye, 'tis a bit of tough luck, indeed! Fair winds to ye, and may yer future blends be lid-secured!


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [ ]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [ ]:
customer_review = """
This leaf blower is pretty amazing.  It has four settings: candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.
"""

review_template = """
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
# create the second template
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template) # notice that here we have  just one input variable(text)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nFor the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price, and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [ ]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat.invoke(messages)
print(response.content)

```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there", "I think it's worth it for the extra features"]
}
```


In [ ]:
# Inspect the type of the output (is it a dict?)
type(response.content)

str

In [ ]:
# You will get an error by running this line of code because'gift' is not a dictionary 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [ ]:
from langchain.output_parsers.structured import ResponseSchema, StructuredOutputParser

In [ ]:
# 1. Define response schemas
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")
delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days did it take for the product to arrive? If this information is not found, output -1.")
price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any sentences about the value or price, and output them as a comma separated Python list.")

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

In [ ]:
# 2. Create parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

`output_parser` is an object of **StructuredOutputParser** with several methods. Most common of them are:

- **`get_format_instructions()`** – Returns a string of instructions for the LLM, telling it how to format its output (usually in JSON) so that `parse()` can extract the information correctly.

- **`parse(text)`** – Takes the LLM’s output string and converts it into a Python dictionary (or structured object) following the response schemas you defined.

- **`response_schemas`** – Shows the list of schemas you used when creating the parser, useful for reference or debugging.


In [ ]:
# List all methods and attributes
print(dir(output_parser))

['InputType', 'OutputType', '__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_pydantic_core_schema__', '__get_pydantic_json_schema__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__or__', '__orig_bases__', '__parameters__', '__pretty__', '__private_attributes__', '__pydantic_complete__', '__pydantic_computed_fields__', '__pydantic_core_schema__', '__pydantic_custom_init__', '__pydantic_decorators__', '__pydantic_extra__', '__pydantic_fields__', '__pydantic_fields_set__', '__pydantic_generic_metadata__', '__pydantic_init_subclass__', '__pydantic_on_complete__', '__pydantic_parent_namespace__', '__pydantic_post_init__', '__pydantic_private__', '__pydantic_root_model__',

In [ ]:
# help(output_parser)

In [ ]:
# 3. Get LLM format instructions
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product to arrive? If this information is not found, output -1.
	"price_value": string  // Extract any sentences about the value or price, and output them as a comma separated Python list.
}
```


In [ ]:
parser_schemas = output_parser.response_schemas
print(parser_schemas)

[ResponseSchema(name='gift', description='Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.', type='string'), ResponseSchema(name='delivery_days', description='How many days did it take for the product to arrive? If this information is not found, output -1.', type='string'), ResponseSchema(name='price_value', description='Extract any sentences about the value or price, and output them as a comma separated Python list.', type='string')]


In [ ]:
# 4. Example
review_template_2 = """For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

In [ ]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

text: 
This leaf blower is pretty amazing.  It has four settings: candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and 

In [ ]:
response = chat.invoke(messages)

In [ ]:
response

AIMessage(content='```json\n{\n\t"gift": "True",\n\t"delivery_days": "2",\n\t"price_value": "It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 316, 'total_tokens': 367, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-Cb6vpUrMirHncGT2dYAgwEy6QjjbL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--78e66b52-2857-47b9-8b7e-bee8a746e1a0-0', usage_metadata={'input_tokens': 316, 'output_tokens': 51, 'total_tokens': 367, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning':

In [ ]:
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [ ]:
output_dict = output_parser.parse(response.content)

In [ ]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [ ]:
type(output_dict)

dict

In [ ]:
output_dict.get('delivery_days')

'2'

In this notebook, we explored how to work with **LangChain templates and structured outputs**. Templates allow us to standardize instructions to the LLM, specifying input variables like text and style, so we don't repeat general instructions each time. We can also define structured output formats directly in the prompt, instructing the LLM to return JSON with keys such as gift, delivery_days, and price_value. However, for greater reliability and convenience, LangChain provides structured parsing using tools like ```ResponseSchema``` and ```StructuredOutputParser``` (or Pydantic models in modern versions). These parsers **validate the output, enforce required fields, and automatically convert responses into Python dictionaries**, ensuring consistency and making downstream use of the data straightforward. Together, templates and structured parsing form a robust workflow for building reliable LLM applications.